In [1]:
import tensorflow as tf
import numpy as np

/Users/tab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
cylinder_features = ['radius', 'height']
cylinder_input_columns = [tf.feature_column.numeric_column(k) for k in cylinder_features]

radius_list_train = (2.0-0.5) * np.random.sample(10000)
height_list_train = (2.0-0.5) * np.random.sample(10000)
volume_list_train = np.pi * radius_list_train * radius_list_train * height_list_train
print(np.pi * radius_list_train[0]**2 * height_list_train[0])
print(radius_list_train[0:10])
print(height_list_train[0:10])
print(volume_list_train[0:10])

radius_list_eval = (2.0-0.5) * np.random.rand(1000)
height_list_eval = (2.0-0.5) * np.random.rand(1000)
volume_list_eval = np.pi * radius_list_eval * radius_list_eval * height_list_eval

4.487986855011567
[1.13259086 0.81846649 0.00532214 0.02792872 1.3881582  1.24454228
 0.30539255 1.15743812 0.35881247 1.47073504]
[1.11366763 0.60453153 0.5796033  0.70803274 0.20525432 0.06551384
 0.49903134 0.71511484 0.62611437 0.30121828]
[4.48798686e+00 1.27224466e+00 5.15767404e-05 1.73502330e-03
 1.24256785e+00 3.18788207e-01 1.46215892e-01 3.00968629e+00
 2.53243666e-01 2.04691629e+00]


In [8]:
def print_rmse(model, eval_input_fn):
  metrics = model.evaluate(input_fn = eval_input_fn)
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
  print(metrics)

In [32]:
cylinder_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {cylinder_features[0]: radius_list_train, cylinder_features[1]: height_list_train},
    y = volume_list_train,
    batch_size = 128,
    num_epochs = 10,
    shuffle = True,
    queue_capacity = 1000
)

cylinder_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {cylinder_features[0]: radius_list_eval, cylinder_features[1]: height_list_eval},
    y = volume_list_eval,
    batch_size = 128,
    shuffle = False
)

cylinder_model = tf.estimator.DNNRegressor(hidden_units=[16, 8, 2],
                                          feature_columns = cylinder_input_columns, model_dir = "./cylinder_model")
cylinder_model.train(input_fn = cylinder_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cylinder_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10e95fe48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./cylinder_model/model.ckpt.
INFO:tensor

In [14]:
print_rmse(cylinder_model, cylinder_eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-18-04:23:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cylinder_model/model.ckpt-782
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-18-04:23:43
INFO:tensorflow:Saving dict for global step 782: average_loss = 0.028029464, global_step = 782, label/mean = 1.6470896, loss = 3.503683, prediction/mean = 1.6164135
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 782: ./cylinder_model/model.ckpt-782
RMSE on dataset = 0.16742002964019775
{'average_loss': 0.028029464, 'label/mean': 1.6470896, 'loss': 3.503683, 'prediction/mean': 1.6164135, 'global_step': 782}
